In [12]:
# for processing numbers and files csv
import pandas as pd
import numpy as np

# preprocessing 
from sklearn.preprocessing import MinMaxScaler

# model related
import tensorflow as tf

In [13]:
metadata = pd.read_csv('metadata.csv').fillna(0)
train_labels = pd.read_csv('train_labels.csv')
metadata.head(2)
train_labels.head(2)
### 2. Preprocessing, Feature extraction
# constants
num_mass_values = 1350
num_time_values = 2650
matrix_shape = (num_time_values, num_mass_values)


# Function to create the Time X Mass matrix for a single data sample
def create_matrix(id ):
    
    sample_id, split, derivate, path, _ = metadata.iloc[id]
#     print(path)
    path =  path
    data = pd.read_csv(path)

    label = None
    if (split == 'train'):
        label = train_labels.iloc[id].values[1:]

    # discretization of time and mass
    data.time = (data.time*50).round()
    data.mass = (data.mass*2).round()
    data.time = data.time.astype(int)
    data.mass = data.mass.astype(int)
    
    # scaling intensity
    data.intensity = MinMaxScaler().fit_transform(data.intensity.values.reshape(-1, 1))
    
    # filling matrix
    matrix = np.zeros(matrix_shape)
    matrix[data['time'],data['mass']] = data['intensity']

    return matrix , label

sample_matrix, sample_label= create_matrix(810) ## 0.2 second # max = 1120
sample_matrix[0:100, 100:200],sample_label
# print(sample_matrix[500:1000,np.random.randint(0,100)])

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.61306026e-06, 0.00000000e+00, 3.07327260e-06, ...,
         0.00000000e+00, 0.00000000e+00, 1.02711550e-06],
        ...,
        [7.23434183e-04, 0.00000000e+00, 1.80854797e-04, ...,
         6.66673508e-07, 0.00000000e+00, 2.03058601e-06],
        [6.43769582e-04, 0.00000000e+00, 1.70465994e-04, ...,
         1.72666132e-06, 1.06921512e-06, 0.00000000e+00],
        [5.57027631e-04, 0.00000000e+00, 1.51995793e-04, ...,
         0.00000000e+00, 1.42215992e-06, 0.00000000e+00]]),
 None)

In [14]:
sample_predictions = pd.read_csv("submission_format.csv")
# predictions

In [15]:
preds = []
model = tf.keras.models.load_model('model1.keras')

In [16]:
for i in range(809):
# for i in range(809,1121):
    mat, _ =  create_matrix(i)
    pred = (model.predict(mat.reshape(-1, 2650,1350,1),verbose = 0)).squeeze()
    preds.append(pred)
    

In [ ]:
predictions = pd.DataFrame(np.array(preds),columns=sample_predictions.columns[1:])
# predictions["sample_id"] = sample_predictions["sample_id"]
predictions["sample_id"] = train_labels["sample_id"]


In [ ]:
predictions = predictions.set_index("sample_id")

In [ ]:
predictions.to_csv("train_predictions1.csv")